In [ ]:
### a small notebook for downloading weather data from darkskyapp

In [115]:
api_key = "46b5c0829bf48d4799127300e62b6161"

In [145]:
import requests
import pandas as pd
from dateutil import parser, rrule
from datetime import datetime, time, date
import time
import json

def get_weather(name, lat, long, _datetime):
    """
    Function to return a data frame of hour-level weather data      
    
    Args:
        lat, long (string): location 
    
    Returns:
        Pandas Dataframe with weather data for specified location and date.
    """
    url = "https://api.darksky.net/forecast/{api_key}/{lat},{long},{_datetime}"
    full_url = url.format(api_key=api_key, lat=lat, long=long, _datetime=_datetime)
    # Request data 
    response = requests.get(full_url)#
    # Convert to pandas dataframe (fails if issues with weather station)
    try:    
        data = response.text
        data = json.loads(data)
        data = data['hourly']['data']
        #print(data)
        #print(data[0])
        dataframe = pd.DataFrame(data) #
        dataframe['city'] = name
        dataframe['datetime'] = pd.to_datetime(dataframe['time'],unit='s')
    except Exception as e:
        print("Issue with data:", data)
        return None
    return dataframe

In [146]:
d = get_weather('NYC', 42.3601, -71.0589, 1230786000)

Issue with data daily usage limit exceeded



In [131]:
d.shape

(24, 19)

In [132]:
d[d['precipType'] == 'snow']

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,city
13,5.02,0,-2.44,0.44,clear-day,0.004,0.0002,0.07,snow,1017.14,Clear,15.75,1230832800,1,8.54,301,20.34,8.05,NYC
14,6.17,0,-2.58,0.41,clear-day,0.012,0.0007,0.09,snow,1017.01,Clear,16.93,1230836400,1,9.26,298,18.74,8.42,NYC
15,8.14,0,-3.23,0.39,clear-day,0.013,0.0007,0.07,snow,1018.12,Clear,17.35,1230840000,0,9.22,297,18.50,6.66,NYC


In [133]:
# Generate a list of all of the dates we want data for

In [134]:
start_date = 1230786000 #"2009-01-01T00:00:00:00"
end_date = 1550085600 # feb 2019 "2015-12-31"
#start = parser.parse(start_date)
#end = parser.parse(end_date)
dates = [start_date + (i*24*60*60) for i in range(1, 365*10)]
dates[:10]

[1230872400,
 1230958800,
 1231045200,
 1231131600,
 1231218000,
 1231304400,
 1231390800,
 1231477200,
 1231563600,
 1231650000]

In [135]:
# Create a list of stations here to download data for
stations = {
    'Chicago': [41.50, -87.37],
    'NYC': [42.3601, -71.0589],
    'San Francisco': [37.47,-122.26],
    'Washingto DC': [38.9072, 77.0369]
}
# Set a backoff time in seconds if a request fails
backoff_time = 60*60*24

df = d  #pd.DataFrame
# Gather data for each station in turn and save to CSV.
for city, coord in stations.items():
    print("Working on {}".format(city))
    for date in dates:
        # Print period status update messages
        if date  % 100000 == 0:
            print("Working on date: {} for station {}".format(date, station))
        done = False
        while done == False:
            try:
                weather_data = get_weather(city, coord[0], coord[1], date)
                done = True
            except ConnectionError as e:
                # May get rate limited by Wunderground.com, backoff if so.
                print("Got connection error on {}".format(date))
                print("Will retry in {} seconds".format(backoff_time))
                time.sleep(backoff_time)
        # Add each processed date to the overall data
        df = pd.concat([df, weather_data])

Working on Chicago
{'time': 1230789600, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 17.33, 'apparentTemperature': 10.37, 'dewPoint': 10.32, 'humidity': 0.73, 'pressure': 1028.14, 'windSpeed': 4.57, 'windGust': 7, 'windBearing': 165, 'cloudCover': 0.1, 'uvIndex': 0, 'visibility': 10}
{'time': 1230876000, 'summary': 'Partly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 30.77, 'apparentTemperature': 23, 'dewPoint': 23.12, 'humidity': 0.73, 'pressure': 1008.84, 'windSpeed': 8.57, 'windGust': 12.5, 'windBearing': 227, 'cloudCover': 0.4, 'uvIndex': 0, 'visibility': 10}
{'time': 1230962400, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 20.6, 'apparentTemperature': 20.6, 'dewPoint': 12.52, 'humidity': 0.7, 'pressure': 1018.24, 'windSpeed': 0.84, 'windGust': 2.86, 'windBearing': 7, 'cloudCover': 0.05, 'uvIndex': 0, 'visibili

{'time': 1232949600, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 8.36, 'apparentTemperature': 8.36, 'dewPoint': -0.64, 'humidity': 0.66, 'pressure': 1030.24, 'windSpeed': 2.82, 'windGust': 5.45, 'windBearing': 271, 'cloudCover': 0.97, 'uvIndex': 0, 'visibility': 10}
{'time': 1233036000, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 14.41, 'apparentTemperature': 14.41, 'dewPoint': 2.34, 'humidity': 0.58, 'pressure': 1032.32, 'windSpeed': 2.22, 'windGust': 4.26, 'windBearing': 289, 'cloudCover': 0.16, 'uvIndex': 0, 'visibility': 10}
{'time': 1233122400, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 18.97, 'apparentTemperature': 13.72, 'dewPoint': 13.42, 'humidity': 0.79, 'pressure': 1018.66, 'windSpeed': 3.53, 'windGust': 6.19, 'windBearing': 13, 'cloudCover': 1, 'uvIndex': 0, 'visibility': 7.54}
{'time': 1233208800, '

{'time': 1235109600, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 19.79, 'apparentTemperature': 10.55, 'dewPoint': 7.61, 'humidity': 0.58, 'pressure': 1019.82, 'windSpeed': 7.28, 'windGust': 14.06, 'windBearing': 310, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 10}
{'time': 1235196000, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 23.24, 'apparentTemperature': 17.69, 'dewPoint': 13.42, 'humidity': 0.66, 'pressure': 1018.13, 'windSpeed': 4.21, 'windGust': 6.47, 'windBearing': 125, 'cloudCover': 0.1, 'uvIndex': 0, 'visibility': 10}
{'time': 1235282400, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 22.05, 'apparentTemperature': 12.7, 'dewPoint': 15.87, 'humidity': 0.77, 'pressure': 1018.47, 'windSpeed': 8.04, 'windGust': 15.98, 'windBearing': 275, 'cloudCover': 1, 'uvIndex': 0, 'visibility': 9.31}
{'time': 1235368800,

{'time': 1237352400, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 59.49, 'apparentTemperature': 59.49, 'dewPoint': 44.75, 'humidity': 0.58, 'pressure': 1016.03, 'windSpeed': 9, 'windGust': 13.63, 'windBearing': 229, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 10}
{'time': 1237438800, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 41.98, 'apparentTemperature': 41.98, 'dewPoint': 30.08, 'humidity': 0.62, 'pressure': 1022.84, 'windSpeed': 2.52, 'windGust': 4.11, 'windBearing': 278, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 10}
{'time': 1237525200, 'summary': 'Partly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 28.77, 'apparentTemperature': 21.57, 'dewPoint': 20.27, 'humidity': 0.7, 'pressure': 1031.2, 'windSpeed': 7.05, 'windGust': 13.13, 'windBearing': 31, 'cloudCover': 0.5, 'uvIndex': 0, 'visibility': 10}
{'time':

{'time': 1239512400, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 35.61, 'apparentTemperature': 32.36, 'dewPoint': 25.93, 'humidity': 0.67, 'pressure': 1027.36, 'windSpeed': 3.9, 'windGust': 6.27, 'windBearing': 28, 'cloudCover': 0.02, 'uvIndex': 0, 'visibility': 10}
{'time': 1239598800, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 45.59, 'apparentTemperature': 40.71, 'dewPoint': 23.11, 'humidity': 0.41, 'pressure': 1020.65, 'windSpeed': 9.63, 'windGust': 14.94, 'windBearing': 101, 'cloudCover': 0.1, 'uvIndex': 0, 'visibility': 10}
{'time': 1239685200, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 39.19, 'apparentTemperature': 33.94, 'dewPoint': 37.38, 'humidity': 0.93, 'pressure': 1008.61, 'windSpeed': 7.38, 'windGust': 10.89, 'windBearing': 54, 'cloudCover': 1, 'uvIndex': 0, 'visibility': 3.71}
{'time': 123977160

...

{'time': 1306472400, 'summary': 'Mostly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 45.65, 'apparentTemperature': 41.08, 'dewPoint': 41.31, 'humidity': 0.85, 'pressure': 1014.84, 'windSpeed': 8.91, 'windGust': 12.76, 'windBearing': 5, 'cloudCover': 0.89, 'uvIndex': 0, 'visibility': 10}
{'time': 1306558800, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 53.26, 'apparentTemperature': 53.26, 'dewPoint': 49.5, 'humidity': 0.87, 'pressure': 1011.61, 'windSpeed': 2.11, 'windGust': 3.69, 'windBearing': 162, 'cloudCover': 1, 'uvIndex': 0, 'visibility': 10}
{'time': 1306645200, 'summary': 'Mostly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 59.04, 'apparentTemperature': 59.07, 'dewPoint': 57.35, 'humidity': 0.94, 'pressure': 1010.5, 'windSpeed': 0.95, 'windGust': 1.87, 'windBearing': 61, 'cloudCover': 0.86, 'uvIndex': 0, 'visibil

{'time': 1308632400, 'summary': 'Mostly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 73.99, 'apparentTemperature': 75.04, 'dewPoint': 68.86, 'humidity': 0.84, 'pressure': 1007.7, 'windSpeed': 7.29, 'windGust': 8.32, 'windBearing': 156, 'cloudCover': 0.86, 'uvIndex': 0, 'visibility': 10}
{'time': 1308718800, 'summary': 'Partly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 70.32, 'apparentTemperature': 71.16, 'dewPoint': 66.39, 'humidity': 0.87, 'pressure': 1004, 'windSpeed': 4.96, 'windGust': 8.58, 'windBearing': 171, 'cloudCover': 0.56, 'uvIndex': 0, 'visibility': 10}
{'time': 1308805200, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 66.49, 'apparentTemperature': 66.54, 'dewPoint': 59.65, 'humidity': 0.79, 'pressure': 1003.52, 'windSpeed': 6.74, 'windGust': 10.48, 'windBearing': 215, 'cloudCover': 1, 'uvIndex': 0, 'visibi

{'time': 1310792400, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 71.72, 'apparentTemperature': 72.06, 'dewPoint': 62.87, 'humidity': 0.74, 'pressure': 1015.84, 'windSpeed': 0.16, 'windGust': 0.79, 'windBearing': 59, 'cloudCover': 0.19, 'uvIndex': 0, 'visibility': 10}
{'time': 1310878800, 'summary': 'Partly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 74.8, 'apparentTemperature': 75.52, 'dewPoint': 66.49, 'humidity': 0.75, 'pressure': 1018.49, 'windSpeed': 0.32, 'windGust': 0.65, 'windBearing': 168, 'cloudCover': 0.28, 'uvIndex': 0, 'visibility': 10}
{'time': 1310965200, 'summary': 'Humid', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 82.43, 'apparentTemperature': 88.83, 'dewPoint': 74.18, 'humidity': 0.76, 'pressure': 1017.89, 'windSpeed': 4.49, 'windGust': 5.73, 'windBearing': 218, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 7.99}


{'time': 1312952400, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 70.29, 'apparentTemperature': 70.43, 'dewPoint': 61.1, 'humidity': 0.73, 'pressure': 1006.52, 'windSpeed': 4.39, 'windGust': 8.14, 'windBearing': 280, 'cloudCover': 0.18, 'uvIndex': 0, 'visibility': 10}
{'time': 1313038800, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 61.67, 'apparentTemperature': 61.67, 'dewPoint': 55.71, 'humidity': 0.81, 'pressure': 1013.31, 'windSpeed': 1.2, 'windGust': 1.65, 'windBearing': 184, 'cloudCover': 0.08, 'uvIndex': 0, 'visibility': 10}
{'time': 1313125200, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 67.3, 'apparentTemperature': 67.3, 'dewPoint': 57.94, 'humidity': 0.72, 'pressure': 1016.43, 'windSpeed': 2.86, 'windGust': 3.95, 'windBearing': 199, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 10}
{'time': 1313211600,

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [136]:
df.shape

(23404, 20)

In [137]:
# Finally combine all of the individual days and output to CSV for analysis.
df.to_csv("weather2009_2019.csv")

In [138]:
df.head()

,apparentTemperature,city,cloudCover,datetime,dewPoint,humidity,icon,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,-5.44,NYC,0.0,2009-01-01 05:00:00,-3.66,0.59,clear-night,NaN,0.0,0.0,NaN,1007.61,Clear,7.68,1230786000,0,6.06,313.0,21.29,8.77
1,-7.07,NYC,0.0,2009-01-01 06:00:00,-4.57,0.58,clear-night,NaN,0.0,0.0,NaN,1008.69,Clear,6.89,1230789600,0,8.23,311.0,21.65,9.58
2,-7.88,NYC,0.0,2009-01-01 07:00:00,-4.99,0.58,clear-night,NaN,0.0,0.0,NaN,1009.97,Clear,6.44,1230793200,0,8.50,315.0,21.07,9.90
3,-8.36,NYC,0.0,2009-01-01 08:00:00,-5.65,0.58,clear-night,NaN,0.0,0.0,NaN,1010.99,Clear,5.86,1230796800,0,8.85,313.0,20.02,9.60
4,-8.03,NYC,0.0,2009-01-01 09:00:00,-6.16,0.58,clear-night,NaN,0.0,0.0,NaN,1011.97,Clear,5.43,1230800400,0,8.37,307.0,19.39,8.57
